In [25]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity 
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional,Dense,GlobalMaxPooling1D,LSTM,Embedding
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [2]:
data = pd.read_csv("../../Data/clean_data.csv")

In [5]:
bert_preproccessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# Using a smaller model replace it with a bigger model
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

In [15]:
def create_embedding(sentence):
    preroccessed_sentence = bert_preproccessor(sentence)
    bert_embedding = bert_model(preroccessed_sentence)
    return bert_embedding["sequence_output"]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(data["Tweet Text"],data["Informativeness"])

In [16]:
X_trarin_embedding = create_embedding(X_train)

In [26]:
X_trarin_embedding.shape

TensorShape([750, 128, 128])

In [27]:
model = keras.Sequential() 
model.add(Bidirectional(LSTM(10,return_sequences=True)))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(5, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",)
model.fit(X_trarin_embedding,y_train,epochs=5)

Epoch 1/5
24/24 [==============================] - 21s 21ms/step - loss: 0.5825
Epoch 2/5
24/24 [==============================] - 0s 17ms/step - loss: 0.3978
Epoch 3/5
24/24 [==============================] - 0s 17ms/step - loss: 0.3183
Epoch 4/5
24/24 [==============================] - 0s 17ms/step - loss: 0.2788
Epoch 5/5
24/24 [==============================] - 0s 17ms/step - loss: 0.2673


In [29]:
# Defining the model

# input = keras.Input(shape=(),dtype="string",name="input")

# layer = Bidirectional(LSTM(128,return_sequences=True))(create_embedding(input))
# layer = Bidirectional(LSTM(128,return_sequences=True))(layer)
# layer = GlobalMaxPooling1D()(layer)
# layer = Dense(512, activation="relu")(layer)
# layer = Dense(1, activation="sigmoid")(layer)

# model = keras.Model(input=[input],output=[layer])
model = keras.Sequential()
model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
# model.add(GlobalMaxPooling1D())
model.add(Dense(512, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(X_trarin_embedding,y_train,epochs=5)
model.summary()

Epoch 1/5


ValueError: in user code:

    C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\keras\engine\training.py:835 run_step  **
 